In [94]:
import os
import pandas as pd
import numpy as np
import xgboost

train=pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/train_features.csv',index_col=0)
test=pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/test_features.csv',index_col=0)

In [95]:
train[0:3]

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev,bandgap_energy_ev_mean,formation_energy_ev_natom_mean
0,1,33,80.0,0.6250,0.3750,0.0,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387,0.787009,0.787009
1,2,194,80.0,0.6250,0.3750,0.0,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210,0.565737,0.565737
2,3,227,40.0,0.8125,0.1875,0.0,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438,0.389779,0.389779


In [96]:
test[0:3]

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,bandgap_energy_ev_mean,formation_energy_ev_natom_mean
0,1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006,0.787009,0.787009
1,2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015,0.787009,0.787009
2,3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,120.0014,1.063387,1.063387


In [97]:
ignore_cols=['id','formation_energy_ev_natom','bandgap_energy_ev']

predictors=[x for x in train.columns if x not in ignore_cols]

In [98]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())


In [99]:
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.decomposition import PCA


# vector = np.vstack((train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values,
#                     test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values))

# pca = PCA().fit(vector)
# train['vector_pca0'] = pca.transform(train[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]
# test['vector_pca0'] = pca.transform(test[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]

# predictors.append('vector_pca0')



Y_feen = np.log(train['formation_energy_ev_natom']+1)
Y_bee = np.log(train['bandgap_energy_ev']+1)


X=train[predictors].as_matrix()
test_mat=test[predictors].as_matrix()

In [100]:
def train_models(X,Y,test,k_folds):

    oof = np.empty(len(X))
    sub_preds = np.zeros(len(test))

    kf = KFold(n_splits=k_folds)
    kf.get_n_splits(X)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        estimator = XGBRegressor( learning_rate=0.05, n_estimators=1000, max_depth=3,\
                                  min_child_weight=5, gamma=0, subsample=0.85, colsample_bytree=0.85,\
                                  objective= 'reg:linear', seed=30,)
        estimator.fit(X_train,y_train,eval_set=[(X_train, y_train), (X_test, y_test)],\
                      eval_metric='rmse', verbose=False,early_stopping_rounds=20) 

        oof[test_index] = estimator.predict(X_test)


        # Update submission
        sub_preds += estimator.predict(test_mat) / k_folds
    return oof, sub_preds

In [101]:
oof_feen, sub_preds_feen = train_models(X,Y_feen,test_mat,k_folds=10)
rmsle(oof_feen,Y_feen)

0.026458712632948578

In [102]:
oof_bee, sub_preds_bee = train_models(X,Y_bee,test_mat,k_folds=10)
rmsle(Y_bee,oof_bee)

0.04998604012842432

In [103]:
estimator = XGBRegressor( learning_rate=0.05, n_estimators=1000, max_depth=3,\
                          min_child_weight=5, gamma=0, subsample=0.85, colsample_bytree=0.85,\
                          objective= 'reg:linear', seed=30,)
estimator.fit(X,Y_feen) 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=30, silent=True, subsample=0.85)

In [107]:
x_test = pd.DataFrame(X)
x_test.columns = ['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']
x_test

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,33.0,80.0,0.6250,0.3750,0.0000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.787009,0.787009
1,194.0,80.0,0.6250,0.3750,0.0000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.565737,0.565737
2,227.0,40.0,0.8125,0.1875,0.0000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.389779,0.389779
3,167.0,30.0,0.7500,0.0000,0.2500,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,1.063387,1.063387
4,194.0,80.0,0.0000,0.6250,0.3750,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.565737,0.565737
5,227.0,40.0,0.5625,0.4375,0.0000,9.8456,5.7176,14.1244,91.1235,91.3033,30.5730,0.389779,0.389779
6,206.0,80.0,0.0312,0.0938,0.8750,10.1664,10.1664,10.1666,90.0009,90.0008,89.9987,0.768953,0.768953
7,12.0,20.0,0.5000,0.0000,0.5000,12.1350,3.1533,6.2880,89.9997,105.1790,90.0001,0.662021,0.662021
8,206.0,80.0,0.5312,0.2188,0.2500,9.4027,9.4023,9.4025,90.0025,90.0030,89.9978,0.768953,0.768953
9,194.0,80.0,0.4062,0.5938,0.0000,6.2520,6.2520,23.7931,90.0172,89.9994,119.9997,0.565737,0.565737


In [105]:
y_test=pd.DataFrame(Y_feen)

In [106]:
from dsx_ml.ml import save
save(name='NomadXgboost',model=estimator,algorithm_type='Regression',x_test=x_test,y_test=y_test)

{'path': '/user-home/1002/DSX_Projects/Feature Engineering and Model Deployments - API and Batch/models/NomadXgboost/1',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python27/scikit-learn-0.19/Feature%20Engineering%20and%20Model%20Deployments%20-%20API%20and%20Batch/NomadXgboost/1'}